# Download Toronto Neighboorhood Information from Wikipedia and format it accordingly

### Installing, importing and setting necessary parameters

In [1]:
# pip install arcgis
from arcgis.gis import GIS
from arcgis.geocoding import get_geocoders, batch_geocode

# Import packages and set API key
import pandas as pd
import numpy as np

C:\Users\marky\anaconda3\lib\site-packages\arcgis\features\_data\geodataset\utils.py:15: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  pd.datetime,


### Import data from wikipedia and drop rows from dataframe where 'borough' is equal to 'Not assigned'

In [2]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df_CA=pd.read_html(url,header=0)[0]
df_CA.replace('Not assigned',np.NaN,inplace=True)
df_CA.dropna(subset=['Borough'],inplace=True)

### Find coordinates for postal codes and add onto current dataset

In [3]:
postal_codes=df_CA["Postal code"].tolist()

In [4]:
# Set geocode setting for arcgis - sensitive info 
# withheld (i.e., username and password)
gis = GIS("http://www.arcgis.com", "User_Name", "Pass_Word")

In [5]:
# use the first of GIS's configured geocoders
geocoder=get_geocoders(gis)[0]
CA_geo=batch_geocode(postal_codes,source_country="CA")

### Use for loop to create dataframe that stores postal codes and coordinates

In [6]:
## For loop to collect coordinates
Postal_code=[]
Lat=[]
Long=[]
for i in range(len(CA_geo)):
    Postal_code.append(CA_geo[i]['address'])
    Lat.append(CA_geo[i]['location']['x'])
    Long.append(CA_geo[i]['location']['y'])

In [7]:
# Convert collected data into a dataframe
df_Postal_code=pd.DataFrame(Postal_code,columns=['Postal code'])
df_Lat=pd.DataFrame(Lat,columns=['Latitude'])
df_Long=pd.DataFrame(Long,columns=['Longitude'])

In [8]:
# Merge tables to include postal codes and their corrsponding coordinates
df_CA_coords=pd.merge(df_Lat,df_Long,left_index=True,right_index=True)
df_CA_coords_code=pd.merge(df_Postal_code,df_CA_coords,left_index=True,right_index=True)

In [9]:
# Merge tables to include postal codes, boroughs, neighborhoods
# and coordinates (longitude and Latitude)
df_CA_geo=pd.merge(df_CA,df_CA_coords_code)
df_CA_geo

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,-79.335641,43.752935
1,M4A,North York,Victoria Village,-79.311890,43.728102
2,M5A,Downtown Toronto,Regent Park / Harbourfront,-79.353041,43.650964
3,M6A,North York,Lawrence Manor / Lawrence Heights,-79.451211,43.723265
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,-79.389390,43.661790
...,...,...,...,...,...
98,M8X,Etobicoke,The Kingsway / Montgomery Road / Old Mill North,-79.509766,43.653340
99,M4Y,Downtown Toronto,Church and Wellesley,-79.381472,43.666659
100,M7Y,East Toronto,Business reply mail Processing CentrE,-79.385450,43.648700
101,M8Y,Etobicoke,Old Mill South / King's Mill Park / Sunnylea /...,-79.493017,43.632798
